# Code refactoring
## Introduction
Most of our code work is not vanilla generation, but changing existing code.

## Installation

In [1]:
%pip install -q openai
# We import the library
from openai import OpenAI

# We instantiate an OpenAI client
client = OpenAI()


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


## Ask the LLM to refactor 

Let's have it write a Hello world in Javascript !

In [2]:
model = "gpt-4o-mini"

existing_code = """
// These functions help print the answers
function my_function() {
    helloWorld();
}

function helloWorld() {
    console.log("Hello, World!");
}
"""

system_prompt = """You are a really good javascript developer. You return code for refactoring. Please only return the code and nothing else. Do not add any comments or explanations. Do not add any extra text. Just return the code in a markdown block."""

question = "make the function helloWorld take a variable and print that instead"

markdown_prompt = question + ". Only return markdown. Don't add any comments or explanations. Just the code. No other text. No other characters. No other symbols. No other lines. No other words. Just the code. Nothing else. Just the code. Nothing else. Just the code. Nothing else. Just the code. Nothing else. Just the code. Nothing else."

# The question we're going to ask
refactor_prompt =  """Given the following piece of code:
{existing_code}

Please refactor it to make it better. The question is:
{question}
"""

# Next we create the completion
completion = client.chat.completions.create(model=model, temperature=0,
    messages=[
        { "role": "system", "content": system_prompt },
        {  "role": "user", "content": refactor_prompt.format(
            existing_code=existing_code,
            question=question
        )}
    ]
)

refactor_result = completion.choices[0].message.content
print(refactor_result)

```javascript
function myFunction() {
    helloWorld("Hello, World!");
}

function helloWorld(message) {
    console.log(message);
}
```


It left out the comments and also change the case of the functionName.

## Diff as an alternative format

In [3]:
# The question we're going to ask
diff_prompt =  """Given the following piece of code:
{existing_code}

Please return the changes to the existing code as a diff. The question is:
{question}
"""

# Next we create the completion
completion = client.chat.completions.create(model=model, temperature=0,
    messages=[
        { "role": "system", "content": system_prompt },
        {  "role": "user", "content": diff_prompt.format(
            existing_code=existing_code,
            question=question
        )}
    ]
)

refactor_result = completion.choices[0].message.content
print(refactor_result)

```diff
- function helloWorld() {
-     console.log("Hello, World!");
+ function helloWorld(message) {
+     console.log(message);
}

-     helloWorld();
+     helloWorld("Hello, World!");
```
